# Pierwszy checkpoint - walidacja Fake News Dataset

### Wojtek Grabias, Wiktor Wierzchowski

In [2]:
import pandas as pd

df1 = pd.read_csv('original_data.csv')

In [3]:
df1

,Unnamed: 0,title,text,Ground Label
0,0,Ann Coulter Make Believes She Has ‘Gay Friend...,"It s hard to believe, but Donald Trump does ha...",fake
1,1,Rating: Moody‘s verbessert Ausblick für Russla...,bankensektor Der russische Staat werde die Ban...,fake
2,2,CAN WE ADD DIRTY MONEY ‘LAUNDERING’ To The Oba...,A member of the House Intelligence Committee i...,fake
3,3,Republicans on Obamacare repeal: 'We're going ...,WASHINGTON (Reuters) - House of Representative...,true
4,4,"Trump, on possible DACA deal, says border wall...",WASHINGTON (Reuters) - U.S. President Donald T...,true
...,...,...,...,...
69040,69040,Burundi opposition platform boycotts new round...,NAIROBI (Reuters) - Burundi s main opposition ...,true
69041,69041,Hillary’s Message To Former Miss Universe Cal...,Miss Universe 1996 Alicia Machado is now an Am...,fake
69042,69042,Cop Crashes Car And Runs Away When More Cops A...,The Daily Sheeple – by Ryan Banister \r\nAn aw...,fake
69043,69043,Trump Stole An Idea From North Korean Propaga...,Jesus f*cking Christ our President* is a moron...,fake


---
## Trzeba też zrobić jakieś podsumowanie tego budowania w kontekście technicznym
##### To co zrobili jest zrealizowane poprawnie i robi to, co założyli, że robi
##### Podzielili poprawnie dane
##### Dobre tam zapoznanie z ramką (chociażby te języki)


In [ ]:
# TUTAJ BĘDZIE KOD KTÓRY ROBI TO CO PISZE NIŻEJ ALE GO NIE DALI WIEC BEDE SZUKAŁ

In [ ]:
# ----------------------------------------------------
# Kod jest tutaj, wystarczy wywołać Main'a
# Main jest podzielony na etapy więc można go sobie tam zmieniać fajnie
# Ale tam dużo działania na jakichś csv to chyba średnio to w notebooku robić
# Po prostu możemy powiedzieć no ta, działa bez kitu
# https://github.com/kitqu/ML-projekt1/blob/main/cleaning_data.py

Po wstępnej analizie danych usunięcie języków innych niż angielski wydaje się w pełni uzasadnione.  
Nie jesteśmy przekonani natomiast czy łączenie kolumn 'text' i 'title', mimo że ułatwi modelowi analize, nie pozbawia nas pewnych informacji.

In [ ]:
# ----------------------------------------------------
# Też jestem tego zdania

Wydaje się, że wszystkie najważniejsze elementy przetworzenia tekstu na potrzeby klasyfikacji zostały uwzględnione. 
Jedyne co sugerowalibyśmy dodać to rozwinięcie potencjalnych skrótów obecnych w języku angielskim. np 'm, 're, 't, ain't itd. 

In [ ]:
# ----------------------------------------------------
# Nawet bym je policzył w kolumnie "text" (konkretniej - ilość "skrótów" w stosunku
# do długości tekstu), ale uwaga na " 's " - nie zamieniałbym (określa przynależność, w dużej ilości
# przypadków źle zamieni)

 
Zastanawia nas jednak czy poprawnym w kontekście analizy fake newsów jest standaryzacja wielkości liter. Jesteśmy skłonni uznać, że nadmierne stosowanie CapsLock'a, szczególnie w tytułach artykułów, może być uznane za potencjalny predyktor nierzetelności (stąd sugestia pozostawienia title i text oddzielnie).

In [ ]:
# ----------------------------------------------------
# Zgadzam się
# Może też dodać stosunek słów napisanych CapsLockiem do długości tytułu
# Albo ogólnie wielkich liter do długości tytułu

In [ ]:
# ----------------------------------------------------
# Czyli ogólnie pomysł jest
# Zostawienie podziału
# Skróty dla kolumny text
# CapsLock dla kolumny title

# Mój pomysł: Wstawić dwie kolumny:
# Uppercase_ratio: stosunek ilości wielkich liter do długości tytułu
# Abbrev_ratio: stosunek ilości skrótów do długości tekstu

# A następnie je wszystkie sprowadzić do parteru tak jak oni robili - w  ten sposób dostaniemy info
# na temat samych słów, a nie to, jak zostały napisane (przestaniemy już odróżniać GÓWNA od gowno)
# ale o to nam w sumie chodzi, po co dalej je traktować jako osobne byty skoro już mamy zapisaną informacje
# o tym, że GÓWNA jest capslockiem.

In [ ]:
# ----------------------------------------------------
# Liczy uppercase
def n_upper_chars(string):
    return sum(map(str.isupper, string))
# Liczy stosunek
def upper_ratio(string):
    return n_upper_chars(string)/len(string.replace(" ",""))

In [ ]:
# ----------------------------------------------------
# Liczenie skrótów zajebałem nie bo po co koło na nowo odkrywać

In [ ]:
# ----------------------------------------------------
contractions = {"ain't": ["am not", "are not", "is not", "has not", "have not"], "aren't": ["are not", "am not"], "i've": ["i have", ]} # ...

# with re you can define advanced regexes, but maybe
# from string import punctuation (suggestion from Martijn Pieters answer
# is still enough for you)
import re

def abbreviation_counter(input_text, abbreviation_dict):   
    count = 0
    # what you want is a list of words. str.split() does this job for you.
    # " " is default and you can also omit this. But if you really need better
    # methods (see answer text abover), you have to take a word tokenizer tool
    # or have to write your own.
    for word in input_text.split(" "):
        # and also clean word (remove ',', ';', ...) afterwards. The advantage of 
        # using re over `from string import punctuation` is that you have more
        # control in what you want to remove. That means that you can add or
        # remove easily any punctuation mark. It could be very handy. It could be
        # also overpowered. If the latter is the case, just stick to Martijn Pieters
        # solution.
        if re.sub(',|;', '', word).lower() in abbreviation_dict:
            count += 1

    return count

In [ ]:
# ----------------------------------------------------
# No i te powyższe funkcje apply damy na te kolumny tworząc nowe, powinno grać i buczeć

In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk import word_tokenize
# funkcja odpowiedzialna za czyszczenie pojedynczego wiersza

def clean_text(text, punctuation_chars):
    # usuwamy zamianę na małe litery
    # text = text.lower()
    # rozwinięcie skrótów
    text = expand_contractions(text)
    # remove punctuation
    text = text.translate(str.maketrans('', '', ''.join(punctuation_chars)))
    # remove digits
    text = text.translate(str.maketrans('', '', string.digits))
    # remove all single characters
    pattern = r'(^| ).( |$)'
    text = re.sub(pattern, ' ', text)
    # remove multiple spaces
    text = re.sub(' +', ' ', text)
    # remove stopwords
    text = delete_stopwords(text)
    # stemming
    text = stemming(text)
    return text

# funkcja odpowiedzialna za usuwanie stopwordów

def delete_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# funkcja odpowiedzialna za stemizację

def stemming(text):
    words = word_tokenize(text)
    porter = PorterStemmer()
    stem_words = [porter.stem(word) for word in words]
    return ' '.join(stem_words)

# funkcja odpowiedzialna za rozwijanie skrótów (zajebana z internetów, nie mam pojęcia o co chodzi z "match")
contraction_dict = {"ain't": "are not","'s":" is","aren't": "are not"}
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(text, contraction_dict = contraction_dict)
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

In [ ]:
# TUTAJ UŻYCIE TEGO ^^^ POKAZANIE: ŚMIGA BAJLANDO ESSUNIA

## brudno pis 

In [ ]:
contraction_dict = {"ain't": "are not","'s":" is","aren't": "are not"}

contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(text,contractions_dict=contractions_dict):
    
    def replace(match):
        
        return contractions_dict[match.group(0)]
    
    return contractions_re.sub(replace, text)

data['reviews.text']=data['reviews.text'].apply(lambda x:expand_contractions(x))

# expand contraction function uses a regular expression to map the contraction to the word. 
# It will match the word with keys and if it is present then replace that word with its value.

Notka do wojtusia: Nie wiem dokładnie co dalej. Jak masz już ogarnięte dane textowe to musisz zamienić je na liczby i do tego można użyć kilku metod. Oni użyli tej tf-idf... no i jakby git ale ciężko powiedzieć czy coś innego było by lepsze zanim nie sprawdzisz tego modelem. Znalazłem, że można jeszcze pierdolnąć bag-of-words(to jest łatwe chyba) albo word embeddings(tego nie patrzyłem).

In [ ]:
# ----------------------------------------------------
# Nie wiem na ile bag-of-words jest fajne (to trochę taki one-hot, wchuj kolumn będzie)

In [ ]:
# ----------------------------------------------------
# Tutaj implementacja word embeddings
# Jeszcze tego nie testowałem ale działa na 99% bo to z dokumentacji zajebałem

In [ ]:
# ----------------------------------------------------
'''
import gensim

sentences = [nltk.word_tokenize(text) for text in df['text']] 
model = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=1, workers=4)

vector = []
for text in df['text']:
    words = nltk.word_tokenize(text)
    vectors = [model.wv[word] for word in words if word in model.wv.vocab]
    if len(vectors) > 0:
        vector.append(np.mean(vectors, axis=0))
    else:
        vector.append(np.zeros(model.vector_size))
vector = np.array(vector)
# Oni też zostawiali jako wektor więc wyjebane niech się męczą
'''

Także zostawiam to na dziś, popatrz, pomyśl, na moje to jeśli chodzi o to co na górze, to to co napisałem to jedyne do czego można się przyjebać. Też daj feedback czy nie jest to głupie z tymi kolumnami i dużymi literkami. 

In [ ]:
# ----------------------------------------------------
# <3